#### Load model keras

In [ ]:
import numpy as np
import keras
from keras.models import load_model

model = load_model('facenet_keras.h5')

#### Train ảnh ra json: danh sách các vector của ảnh train

In [ ]:
from os import listdir
from os.path import isfile, join
import json
import numpy as np
import cv2
from scipy.spatial import distance

In [ ]:
def detect_faces(image, margin):
    '''
    sử dụng detectMultiScale của opencv để detect ra tọa độ các khuôn mặt từ ảnh image
    :return: danh sách các khuôn mặt
    '''
    cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')
#     faces = cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=3)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = cascade.detectMultiScale(gray, 1.1, 4)

    aligned_images = []
    margin_half = margin // 2
    
    for face in faces:
        (x, y, w, h) = face
        cropped = image[y - margin_half:y + h + margin_half, x - margin_half:x + w + margin_half,:]
        aligned = cv2.resize(cropped, (160, 160))
        aligned_images.append(aligned)

    return faces, np.array(aligned_images)

In [ ]:
def prewhiten(x):
    '''
    tiền xử lý khuôn mặt trước khi đưa vào mô hình
    '''
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0 / np.sqrt(size))
    y = (x - mean) / std_adj
    return y

In [ ]:
def l2_normalize(x, axis=-1, epsilon=1e-10):
    '''
    chuẩn hóa vector đầu ra
    '''
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output

In [ ]:
def calc_embs(image, margin=10):
    '''
    xử lý ảnh, đưa ảnh về vector 
    '''
    faces, aligned_images = detect_faces(image, margin)
    if len(faces) == 0:
        return None, None

    prewhiten_images = prewhiten(aligned_images)

    predicts = model.predict_on_batch(prewhiten_images)

    embs = []
    for predict in predicts:
        embs.append(l2_normalize(predict).tolist())

    return faces, embs

In [ ]:
def cal_distance(vector1, vector2):
    '''
    tính khoảng cách giữa 2 vector
    '''
    return distance.euclidean(vector1, vector2)

In [ ]:
# lấy toàn bộ thư mục trong Train, mỗi thư mục là một nhãn
# trong mỗi thư mục chứa danh sách các ảnh cần train
train_names = listdir('Train')

data = []

for name in train_names:
    arr = []
    path = join('Train', name)
    files = [f for f in listdir(path) if isfile(join(path, f))]
    for file in files:
        filepath = join(path, file)

        # đọc ảnh xong tính vector đầu ra, đưa vào danh sách
        image = cv2.imread(filepath)
        _, embs = calc_embs(image)

        # khi train thì mỗi ảnh chỉ có một khuôn mặt nên chỉ lấy phần từ đầu trong ds (embs[0])
        arr.append(embs[0])

    # mỗi khuôn mặt sẽ bao gồm nhãn (tên) và danh sách các vector đi kèm
    data.append({'name': name, 'values': arr})

data = {'data': data}

with open('data.json', 'w') as file:
    json.dump(data, file, indent=4)

#### Test

In [ ]:
def paint(image, box, label):
    '''
    vẽ khung và thêm nhãn cho bức ảnh theo box và label
    '''
    color_box = [0, 192, 0] # màu xanh làm màu khung
    font = cv2.FONT_HERSHEY_PLAIN
    x, y, w, h = box
    text_width = cv2.getTextSize(label, font, 1.2, 2)[0][0]
    cv2.rectangle(image, (x, y), (x + w, y + h), color_box, 2)
    cv2.putText(image, label, (x + 10, y + 25), font, 1.2, (255, 255, 255), 1, cv2.LINE_AA)

In [ ]:
def predict(image):
    '''
    dự đoán tên người trong bức ảnh được truyền vào
    '''
    # trước tiên mở file data.json để lấy dữ liệu đã train: các khuôn mặt kèm theo các vector
    with open('data.json') as json_file:
        data = json.load(json_file)['data']

    # lấy ra tất cả các box của ảnh
    boxes, embs = calc_embs(image)
    
    if boxes is None:
        return None

    result = []
    for box, emb in zip(boxes, embs):
        min_value = 99999
        label = None

        # quét toàn bộ vector đã lấy từ file json, xem vector nào gần với khuôn mặt này nhất
        for vectors in data:  # vectors là danh sách các tên người kèm các vector
            for vector in vectors['values']:  # mỗi tên lại có nhiều vector (khuôn mặt)
                dis = cal_distance(vector, emb)
                if dis < min_value:
                    min_value = dis
                    label = vectors['name']
        paint(image, box, label)  # tìm được box và label rồi thì vẽ vào ảnh
        result.append({'box': box.tolist(), 'label': label})

    return result

In [ ]:
%%time

filename = 'Test/test5.jpg'

image = cv2.imread(filename)

print(predict(image))

cv2.imshow('Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()